In [ ]:
import os
import json
import pandas as pd
import datetime
import requests
import numpy as np

In [ ]:
def readSecrets ():
    filename = os.path.join('secrets.json')
    try:
        with open(filename, mode='r') as f:
            return json.loads(f.read())
    except FileNotFoundError:
        return {}

# Dictionary of {"swid": "", "espn_s2": ""}
secrets = readSecrets()
LEAGUE_ID = "699385"

In [ ]:
def reqUsersUrl(year):
    if year < datetime.date.today().year:
        return 'https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/{}?seasonId={}'.format(LEAGUE_ID, str(year))
    else:
        return 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/{}/segments/0/leagues/{}'.format(str(year), LEAGUE_ID)

print(reqUsersUrl(2022), reqUsersUrl(2015))

In [ ]:
r = requests.get(reqUsersUrl(2022), cookies=secrets)
res = r.json()
del res['teams'][1]['owners'][1]
res['teams'][1]['owners']
print(res)

In [ ]:
teams = pd.DataFrame.from_records(res['teams'])
members = pd.DataFrame.from_records(res['members'])
teams['owners'] = [owner[0].strip('[]') for owner in teams['owners']]
teams

In [ ]:
members.head()

In [ ]:
m = requests.get(reqUsersUrl(2022), cookies=secrets, params={'view': 'mMatchup'})
m = m.json()

In [ ]:
m

In [ ]:
listScores = [[
    game['matchupPeriodId'], game['away']['totalPoints'], game['away']['teamId'], game['home']['totalPoints'], game['home']['teamId']
] for game in m['schedule']
]
print(listScores)

In [ ]:
matchups = pd.DataFrame(listScores, columns = ['Week', 'awayPoints', 'awayTeam', 'homePoints', 'homeTeam'])
matchups

In [ ]:
pastYears = list(range(2015,2023))
print(pastYears)

In [ ]:
allScores = []
for year in pastYears:
    req = requests.get(reqUsersUrl(year), cookies=secrets, params={'view': 'mMatchup'})
    for game in m['schedule']:
        allScores.append([ year, game['matchupPeriodId'], game['away']['totalPoints'], game['away']['teamId'], game['home']['totalPoints'], game['home']['teamId']])

print(allScores)
    

In [ ]:
allMatchups = pd.DataFrame(allScores, columns = ['Year', 'Week', 'awayPoints', 'awayTeam', 'homePoints', 'homeTeam'])

In [ ]:
allMatchups.head(250)

In [ ]:
allMatchups.dtypes


In [ ]:
for year in allMatchups.Year.unique():
    selector = allMatchups.loc[allMatchups['Year'] == year]
    print(selector['homePoints'])
    